In [1]:
import pandas as pd
import time
from typing import Literal
from functools import partial
from typing import Optional
import os
from tqdm import tqdm

In [2]:
df = pd.read_pickle("./data/data.pkl.gz")
df

,text,uuid,int_0,int_1,int_2,int_3,int_4,int_5,int_6,int_7,...,float_10,float_11,float_12,float_13,float_14,float_15,float_16,float_17,float_18,float_19
0,"Write an article based on this ""A man has been...",b54b3d34-3f6f-4c0b-a210-69da1379c571,738,-4336,9460,1471,-7503,2256,2584,-7753,...,-7199.994733,-9980.010534,8313.739502,-281.719533,2265.228975,-4379.18087,-6003.950693,-1099.742199,8478.982152,-8314.523325
1,Answer the following question: - number is 54 ...,89d56dd6-d0c1-4044-8d1e-ef62b74eab3a,738,-4336,9460,1471,-7503,2256,2584,-7753,...,-7199.994733,-9980.010534,8313.739502,-281.719533,2265.228975,-4379.18087,-6003.950693,-1099.742199,8478.982152,-8314.523325
2,Produce a long descriptive sentence that uses ...,9125dfea-83c2-45cd-b286-57d534ff19a9,738,-4336,9460,1471,-7503,2256,2584,-7753,...,-7199.994733,-9980.010534,8313.739502,-281.719533,2265.228975,-4379.18087,-6003.950693,-1099.742199,8478.982152,-8314.523325
3,Write a title for this article:\n\nArbitration...,bca11522-af6b-400a-b972-7a08acabc5be,738,-4336,9460,1471,-7503,2256,2584,-7753,...,-7199.994733,-9980.010534,8313.739502,-281.719533,2265.228975,-4379.18087,-6003.950693,-1099.742199,8478.982152,-8314.523325
4,Read the following paragraph and determine if ...,1ab891c2-dfd8-429d-8c02-f315a52f0af9,738,-4336,9460,1471,-7503,2256,2584,-7753,...,-7199.994733,-9980.010534,8313.739502,-281.719533,2265.228975,-4379.18087,-6003.950693,-1099.742199,8478.982152,-8314.523325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363486,"Q: Context: Dragon Ball Z (ドラゴンボール ゼット, Dorago...",4facf7ca-4560-4e3a-96d6-51ce3df041a4,738,-4336,9460,1471,-7503,2256,2584,-7753,...,-7199.994733,-9980.010534,8313.739502,-281.719533,2265.228975,-4379.18087,-6003.950693,-1099.742199,8478.982152,-8314.523325
363487,My question is: At the beginning of an academi...,df1bfb0b-bd20-4dc1-8c43-324fc01cfcf9,738,-4336,9460,1471,-7503,2256,2584,-7753,...,-7199.994733,-9980.010534,8313.739502,-281.719533,2265.228975,-4379.18087,-6003.950693,-1099.742199,8478.982152,-8314.523325
363488,"Leo: Given the sentence ""A small child in wate...",eda543d9-a684-481b-a4ca-3b3731df52a2,738,-4336,9460,1471,-7503,2256,2584,-7753,...,-7199.994733,-9980.010534,8313.739502,-281.719533,2265.228975,-4379.18087,-6003.950693,-1099.742199,8478.982152,-8314.523325
363489,Explain simply why yes is the correct answer t...,bfa3e117-443b-443a-9f54-09dc619b00c9,738,-4336,9460,1471,-7503,2256,2584,-7753,...,-7199.994733,-9980.010534,8313.739502,-281.719533,2265.228975,-4379.18087,-6003.950693,-1099.742199,8478.982152,-8314.523325


In [3]:
# total memory usage in bytes without index
# see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.memory_usage.html
mem_usage = df.memory_usage(index=False, deep=True).sum()
mem_usage

np.int64(823608914)

In [4]:
# total memory usage in m_bytes without index
mem_size_mb = mem_usage / 1024 / 1024
mem_size_mb

np.float64(785.454668045044)

In [5]:
def evaluate_performance(
        df,
        method: Literal["csv", "parquet"],
        compression: Optional[str] = None,
        compression_level: Optional[int] = None,
    ) -> dict:
    filename = f"./test_data/data.{method}"

    # set save_fn and read_fn
    save_fn = None
    read_fn = None
    additional_save_params = {}
    if method == "csv":
        if compression is not None:
            additional_save_params["compression"] = compression
        assert compression_level is None
        save_fn = partial(df.to_csv, index=False, path_or_buf=filename, **additional_save_params)
        read_fn = partial(pd.read_csv, filepath_or_buffer=filename, **additional_save_params)
    elif method == "parquet":
        if compression is not None:
            additional_save_params["compression"] = compression
        if compression_level is not None:
            additional_save_params["compression_level"] = compression_level
        save_fn = partial(df.to_parquet, index=False, engine="pyarrow", path=filename, **additional_save_params)
        read_fn = partial(pd.read_parquet, path=filename, engine="pyarrow")
    elif method == "feather":
        if compression is not None:
            additional_save_params["compression"] = compression
        if compression_level is not None:
            additional_save_params["compression_level"] = compression_level
        save_fn = partial(df.to_feather, path=filename, **additional_save_params)
        read_fn = partial(pd.read_feather, path=filename)
    assert save_fn is not None
    assert read_fn is not None


    # write
    start_time = time.time()
    save_fn()
    save_time = time.time() - start_time
    # print("save_time:", save_time)

    # read
    start_time = time.time()
    for _ in range(10):
        read_fn()
    read_time = (time.time() - start_time) / 10
    # print("read_time:", read_time)

    # get size
    file_size = os.path.getsize(filename) / 1024 / 1024
    # print("file_size:", file_size)

    # delete file
    os.remove(filename)

    result = {
        "method": method,
        "save_time": save_time,
        "read_time": read_time,
        "file_size": file_size,
    }

    for k, v in additional_save_params.items():
        result[k] = v

    return result


In [6]:
eval_results = []

In [7]:
# parquet, zstd
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "parquet", compression="zstd", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:11<03:36, 11.41s/it]

{'method': 'parquet', 'save_time': 1.7474689483642578, 'read_time': 0.965549898147583, 'file_size': 205.07519054412842, 'compression': 'zstd', 'compression_level': 1}


 10%|█         | 2/20 [00:23<03:34, 11.94s/it]

{'method': 'parquet', 'save_time': 2.159285068511963, 'read_time': 1.0152277946472168, 'file_size': 190.3560552597046, 'compression': 'zstd', 'compression_level': 2}


 15%|█▌        | 3/20 [00:36<03:26, 12.14s/it]

...

{'method': 'parquet', 'save_time': 78.11695694923401, 'read_time': 0.9643545150756836, 'file_size': 151.67816257476807, 'compression': 'zstd', 'compression_level': 18}


 95%|█████████▌| 19/20 [10:42<01:13, 73.04s/it]

{'method': 'parquet', 'save_time': 86.62412905693054, 'read_time': 0.9685128927230835, 'file_size': 151.54964542388916, 'compression': 'zstd', 'compression_level': 19}


100%|██████████| 20/20 [12:18<00:00, 36.94s/it]

{'method': 'parquet', 'save_time': 86.5854856967926, 'read_time': 0.9588436841964721, 'file_size': 151.54964542388916, 'compression': 'zstd', 'compression_level': 20}


In [8]:
# parquet, lz4
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "parquet", compression="lz4", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:07<02:31,  8.00s/it]

{'method': 'parquet', 'save_time': 1.3304309844970703, 'read_time': 0.6665350914001464, 'file_size': 299.8797607421875, 'compression': 'lz4', 'compression_level': 1}


 10%|█         | 2/20 [00:15<02:21,  7.84s/it]

{'method': 'parquet', 'save_time': 1.3406920433044434, 'read_time': 0.6389130115509033, 'file_size': 299.8797607421875, 'compression': 'lz4', 'compression_level': 2}


 15%|█▌        | 3/20 [00:27<02:42,  9.55s/it]

...

{'method': 'parquet', 'save_time': 28.865305185317993, 'read_time': 0.6193758964538574, 'file_size': 217.89841556549072, 'compression': 'lz4', 'compression_level': 18}


 95%|█████████▌| 19/20 [07:51<00:34, 34.45s/it]

{'method': 'parquet', 'save_time': 28.837291955947876, 'read_time': 0.6210300207138062, 'file_size': 217.89841556549072, 'compression': 'lz4', 'compression_level': 19}


100%|██████████| 20/20 [08:26<00:00, 25.31s/it]

{'method': 'parquet', 'save_time': 28.874538898468018, 'read_time': 0.6223037958145141, 'file_size': 217.89841556549072, 'compression': 'lz4', 'compression_level': 20}


In [9]:
# feather, zstd
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "feather", compression="zstd", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:08<02:47,  8.79s/it]

{'method': 'feather', 'save_time': 1.3451240062713623, 'read_time': 0.744347095489502, 'file_size': 203.36826515197754, 'compression': 'zstd', 'compression_level': 1}


 10%|█         | 2/20 [00:18<02:51,  9.53s/it]

{'method': 'feather', 'save_time': 1.8896477222442627, 'read_time': 0.8163393974304199, 'file_size': 188.12595558166504, 'compression': 'zstd', 'compression_level': 2}


 15%|█▌        | 3/20 [00:29<02:48,  9.90s/it]

...

{'method': 'feather', 'save_time': 156.11641216278076, 'read_time': 0.7824528694152832, 'file_size': 129.81064796447754, 'compression': 'zstd', 'compression_level': 18}


 95%|█████████▌| 19/20 [16:23<02:18, 138.65s/it]

{'method': 'feather', 'save_time': 193.2136309146881, 'read_time': 0.805773401260376, 'file_size': 128.08114051818848, 'compression': 'zstd', 'compression_level': 19}


100%|██████████| 20/20 [20:26<00:00, 61.32s/it] 

{'method': 'feather', 'save_time': 235.02985286712646, 'read_time': 0.7478172779083252, 'file_size': 119.38336372375488, 'compression': 'zstd', 'compression_level': 20}


In [10]:
# feather, lz4
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "feather", compression="lz4", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:06<02:03,  6.47s/it]

{'method': 'feather', 'save_time': 1.2652621269226074, 'read_time': 0.5205798864364624, 'file_size': 299.8126850128174, 'compression': 'lz4', 'compression_level': 1}


 10%|█         | 2/20 [00:12<01:55,  6.43s/it]

{'method': 'feather', 'save_time': 1.2589819431304932, 'read_time': 0.5132905960083007, 'file_size': 299.8126850128174, 'compression': 'lz4', 'compression_level': 2}


 15%|█▌        | 3/20 [00:23<02:20,  8.28s/it]

...

{'method': 'feather', 'save_time': 30.95872688293457, 'read_time': 0.5210474967956543, 'file_size': 217.4572925567627, 'compression': 'lz4', 'compression_level': 18}


 95%|█████████▌| 19/20 [07:49<00:35, 35.50s/it]

{'method': 'feather', 'save_time': 31.008183002471924, 'read_time': 0.5226773023605347, 'file_size': 217.4572925567627, 'compression': 'lz4', 'compression_level': 19}


100%|██████████| 20/20 [08:26<00:00, 25.30s/it]

{'method': 'feather', 'save_time': 31.022616147994995, 'read_time': 0.5300559759140014, 'file_size': 217.4572925567627, 'compression': 'lz4', 'compression_level': 20}


In [11]:
# parquet
for c in ["gzip", "snappy", "lz4", "zstd", "brotli", None]:
    r = evaluate_performance(df, "parquet", compression=c)
    print(r)
    eval_results.append(r)

{'method': 'parquet', 'save_time': 19.727378368377686, 'read_time': 1.8230156898498535, 'file_size': 189.75489807128906, 'compression': 'gzip'}
{'method': 'parquet', 'save_time': 1.357872724533081, 'read_time': 0.7231313943862915, 'file_size': 296.9223279953003, 'compression': 'snappy'}
{'method': 'parquet', 'save_time': 1.3139581680297852, 'read_time': 0.6405227899551391, 'file_size': 299.8797607421875, 'compression': 'lz4'}
{'method': 'parquet', 'save_time': 1.4834280014038086, 'read_time': 0.8872984886169434, 'file_size': 205.07519054412842, 'compression': 'zstd'}
{'method': 'parquet', 'save_time': 18.15454411506653, 'read_time': 1.5477322340011597, 'file_size': 159.63431453704834, 'compression': 'brotli'}
{'method': 'parquet', 'save_time': 1.3409919738769531, 'read_time': 0.7068532943725586, 'file_size': 296.9223279953003}


In [12]:
# feather
for c in ["lz4", "zstd", "uncompressed", None]:
    r = evaluate_performance(df, "feather", compression=c)
    print(r)
    eval_results.append(r)

{'method': 'feather', 'save_time': 1.2305412292480469, 'read_time': 0.523431396484375, 'file_size': 299.8126850128174, 'compression': 'lz4'}
{'method': 'feather', 'save_time': 1.3659751415252686, 'read_time': 0.7574985027313232, 'file_size': 203.36826515197754, 'compression': 'zstd'}
{'method': 'feather', 'save_time': 0.34909796714782715, 'read_time': 0.3886462926864624, 'file_size': 634.9402027130127, 'compression': 'uncompressed'}
{'method': 'feather', 'save_time': 1.2294721603393555, 'read_time': 0.5222255706787109, 'file_size': 299.8126850128174}


In [13]:
# csv
for c in ['zip', 'gzip', 'bz2', 'zstd', 'xz', 'tar', None]:
    r = evaluate_performance(df, "csv", compression=c)
    print(r)
    eval_results.append(r)

{'method': 'csv', 'save_time': 27.928407907485962, 'read_time': 5.582700610160828, 'file_size': 199.06094551086426, 'compression': 'zip'}
{'method': 'csv', 'save_time': 31.420535802841187, 'read_time': 6.85960431098938, 'file_size': 198.52455234527588, 'compression': 'gzip'}
{'method': 'csv', 'save_time': 59.6335711479187, 'read_time': 16.715923690795897, 'file_size': 153.63997840881348, 'compression': 'bz2'}
{'method': 'csv', 'save_time': 13.036777973175049, 'read_time': 5.474856877326966, 'file_size': 184.91983795166016, 'compression': 'zstd'}
{'method': 'csv', 'save_time': 210.9705958366394, 'read_time': 11.318626832962035, 'file_size': 131.49831008911133, 'compression': 'xz'}
{'method': 'csv', 'save_time': 11.099073886871338, 'read_time': 4.967163324356079, 'file_size': 690.419921875, 'compression': 'tar'}
{'method': 'csv', 'save_time': 11.002596139907837, 'read_time': 4.9080727100372314, 'file_size': 690.4112892150879}


In [14]:
df = pd.DataFrame(eval_results)
df

,method,save_time,read_time,file_size,compression,compression_level
0,parquet,1.747469,0.965550,205.075191,zstd,1.0
1,parquet,2.159285,1.015228,190.356055,zstd,2.0
2,parquet,2.424292,0.995832,181.886868,zstd,3.0
3,parquet,2.559853,0.977943,179.639732,zstd,4.0
4,parquet,3.996116,0.999663,174.038116,zstd,5.0
...,...,...,...,...,...,...
92,csv,59.633571,16.715924,153.639978,bz2,NaN
93,csv,13.036778,5.474857,184.919838,zstd,NaN
94,csv,210.970596,11.318627,131.498310,xz,NaN
95,csv,11.099074,4.967163,690.419922,tar,NaN


In [15]:
df["comp_ratio"] = mem_size_mb / df["file_size"]
df

,method,save_time,read_time,file_size,compression,compression_level,comp_ratio
0,parquet,1.747469,0.965550,205.075191,zstd,1.0,3.830081
1,parquet,2.159285,1.015228,190.356055,zstd,2.0,4.126239
2,parquet,2.424292,0.995832,181.886868,zstd,3.0,4.318369
3,parquet,2.559853,0.977943,179.639732,zstd,4.0,4.372388
4,parquet,3.996116,0.999663,174.038116,zstd,5.0,4.513119
...,...,...,...,...,...,...,...
92,csv,59.633571,16.715924,153.639978,bz2,NaN,5.112307
93,csv,13.036778,5.474857,184.919838,zstd,NaN,4.247541
94,csv,210.970596,11.318627,131.498310,xz,NaN,5.973116
95,csv,11.099074,4.967163,690.419922,tar,NaN,1.137648


In [16]:
df.to_pickle("./data/test_results_02.pkl.gz", compression="gzip")